In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
import numpy as np
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\Quantomatic.jar'
zx.tikz.tikzit_location = r'C:\Users\John\Documents\tikzit\tikzit.exe'

In [3]:
def generate_clifford_circuit(qubits, depth, p_cnot=0.3, p_t=0):
    p_s = 0.5*(1.0-p_cnot-p_t)
    p_had = 0.5*(1.0-p_cnot-p_t)
    c = zx.Circuit(qubits)
    for _ in range(depth):
        r = random.random()
        if r > 1-p_had:
            c.add_gate("HAD",random.randrange(qubits))
        elif r > 1-p_had-p_s:
            c.add_gate("S",random.randrange(qubits))
        elif r > 1-p_had-p_s-p_t:
            c.add_gate("T",random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl!=tgt: break
            c.add_gate("CNOT",tgt,ctrl)
    return c

In [118]:
c = zx.Circuit.load(r'..\..\pytket\examples\benchmarking\ChemistrySet\H2_UCCSD_JW_631g.qasm').split_phase_gates().to_basic_gates()
#c = zx.Circuit.load(r'..\circuits\Fast\barenco_tof_5_before').to_basic_gates()
print(c.stats())
c_opt = zx.optimize.basic_optimization(c).to_basic_gates()
print(c_opt.stats())

Circuit H2_UCCSD_JW_631g.qasm on 8 qubits with 5236 gates.
        84 is the T-count
        5152 Cliffords among which 
        768 2-qubit gates and 1880 Hadamard gates.
Circuit  on 8 qubits with 1042 gates.
        84 is the T-count
        958 Cliffords among which 
        526 2-qubit gates and 213 Hadamard gates.


In [130]:
architecture = zx.routing.architecture.create_rigetti_16q_aspen_architecture()
#architecture.visualize()

In [94]:
architecture = zx.cnot_mapper.create_architecture('square',n_qubits=9)
#architecture.visualize()

In [131]:
seed = 1338
random.seed(seed)
reps = 20
qubits = 20
depth = 800

method1 = 0
method2 = 0
method3 = 0

for i in range(1,reps+1):
    if i%10 == 0: print(i, end='.')
    #c = generate_clifford_circuit(qubits, depth, p_cnot=0.3, p_t=0.2)
    c = zx.Circuit.load(r'..\..\pytket\examples\benchmarking\ChemistrySet\H2_UCCSD_JW_631g.qasm')
    perm = list(range(16))
    random.shuffle(perm)
    c.gates = [g.reposition(perm) for g in c.gates]
    c.qubits = len(perm)
    c_opt = zx.optimize.basic_optimization(c.split_phase_gates().to_basic_gates()).to_basic_gates()
    g = c.to_graph()
    zx.full_reduce(g)
    g.normalise()
    #c2 = zx.extract.streaming_extract(g2,quiet=True).to_basic_gates()
    #c2 = zx.optimize.basic_optimization(c2.to_basic_gates()).to_basic_gates()
    #g2 = g.copy()
    #c3 = zx.extract.modified_extract(g2).to_basic_gates()
    #c3 = zx.optimize.basic_optimization(c3.to_basic_gates()).to_basic_gates()
    g2 = g.copy()
    c2 = mapped_extract(g2,architecture,quiet=True)
    #c2 = zx.extract.modified_extract(g2,optimize_czs=True,optimize_cnots=1,quiet=False).to_basic_gates()
    c2 = zx.optimize.basic_optimization(c2.to_basic_gates(),do_swaps=False).to_basic_gates()
    
    #g2 = g.copy()
    #c2 = zx.extract.modified_extract(g2,optimize_czs=True,optimize_cnots=2,quiet=True).to_basic_gates()
    #c2 = zx.optimize.basic_optimization(c2.to_basic_gates()).to_basic_gates()
    #print()
    #g2 = g.copy()
    #c4 = extract_better(g2,optimize_czs=True,optimize_cnots=3,quiet=False).to_basic_gates()
    #c4 = zx.optimize.basic_optimization(c4.to_basic_gates()).to_basic_gates()
    print(perm, c2.twoqubitcount())
    method1 += c2.twoqubitcount()
    #method2 += c3.twoqubitcount()
    #method3 += c4.twoqubitcount()

print(c_opt.to_basic_gates().stats())
print(method1/reps)#, method2/reps)#, method3/reps)

[2, 3, 1, 0, 6, 12, 5, 14, 9, 8, 15, 7, 10, 13, 4, 11] 2907
[0, 9, 12, 5, 15, 10, 13, 2, 4, 14, 8, 3, 7, 11, 1, 6] 2457
[13, 6, 9, 1, 8, 12, 14, 10, 0, 15, 5, 3, 2, 7, 4, 11] 2711
[10, 8, 11, 0, 5, 4, 6, 3, 12, 7, 2, 1, 15, 13, 14, 9] 1616
[9, 12, 7, 0, 15, 10, 5, 2, 3, 13, 4, 14, 6, 11, 8, 1] 2532
[3, 13, 14, 6, 4, 0, 15, 2, 1, 12, 5, 10, 11, 7, 8, 9] 2674
[10, 2, 8, 0, 4, 1, 14, 7, 15, 12, 3, 9, 6, 13, 5, 11] 3053
[10, 15, 0, 4, 2, 3, 9, 13, 7, 12, 8, 5, 11, 14, 6, 1] 3075
[1, 0, 13, 10, 6, 11, 12, 7, 14, 15, 3, 9, 2, 5, 4, 8] 2557
10.[6, 5, 15, 14, 8, 1, 4, 2, 13, 10, 7, 11, 0, 12, 9, 3] 2482
[7, 4, 10, 13, 11, 8, 1, 0, 12, 15, 3, 9, 5, 14, 6, 2] 2556
[1, 8, 15, 9, 7, 4, 3, 5, 10, 13, 0, 6, 11, 2, 14, 12] 2074
[7, 14, 15, 4, 9, 10, 0, 3, 8, 12, 1, 6, 11, 13, 5, 2] 1965
[7, 15, 6, 9, 13, 5, 4, 14, 8, 10, 11, 3, 1, 12, 0, 2] 2159
[9, 11, 0, 6, 15, 8, 13, 4, 12, 1, 5, 2, 10, 3, 7, 14] 1890
[1, 15, 0, 5, 9, 7, 12, 8, 13, 2, 6, 4, 10, 11, 14, 3] 2719
[7, 2, 6, 15, 5, 8, 12, 3, 0, 11, 14,

In [108]:
%debug

> <ipython-input-105-c56f3abf77e8>(197)mapped_extract()
    195                 good_verts[v] = w
    196         if len(good_verts) <= len(finished):# and len(finished) != len(frontier):
--> 197             raise Exception("No extractable vertex found. Something went wrong")
    198         hads = []
    199         for v,w in good_verts.items(): # Update frontier vertices

ipdb> m2
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 ]
[ 0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 ]
[ 0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0 ]
[ 0  0  0  1  1  0  0  0  0  0  0  0  1  1  1  0  1 ]
[ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 ]
[ 0  0  0  0  1  1  0  0  0  1  0  1  1  0  1  1  1 ]
[ 0  0  0  0  0  1  0  0  0  1  0  1  0  0  0  1  0 ]
[ 0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0 ]
[ 0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0 ]
ipdb> q


In [109]:
for i, r in enumerate(range(2,5)):
    print(i,r)

0 2
1 3
2 4


In [97]:
zx.d3.draw(g2)

In [84]:
zx.compare_tensors(c2,c_opt,False)

True

In [25]:
l = [2,3,1]
l.sort()
l

[1, 2, 3]

In [112]:
from pyzx.extract import max_overlap, bi_adj, connectivity_from_biadj, apply_rule, pivot, permutation_as_swaps
from pyzx.extract import filter_duplicate_cnots
from pyzx.circuit import Circuit, CNOT
from pyzx.simplify import id_simp
from pyzx.linalg import Mat2, greedy_reduction, CNOTMaker, column_optimal_swap
from pyzx.routing.steiner import steiner_gauss

def steiner_cnots(m, architecture):
    cn = CNOTMaker()
    steiner_gauss(m.copy(),architecture, full_reduce=True, x=cn)
    return cn.cnots

def mapped_extract(g, architecture, quiet=True):
    """Given a graph put into semi-normal form by :func:`simplify.full_reduce`, 
    it extracts its equivalent set of gates into an instance of :class:`circuit.Circuit`,
    mapped to the given architecture.

    :param g: The ZX-diagram graph to be extracted into a Circuit.
    :param optimize_czs: Whether to try to optimize the CZ-subcircuits by exploiting overlap between the CZ gates
    :param optimize_cnots: (0,1,2,3) Level of CNOT optimization to apply.
    :param quiet: Whether to print detailed output of the extraction process.
    """
    g.normalise()
    qs = g.qubits() # We are assuming that these are objects that update...
    rs = g.rows()   # ...to reflect changes to the graph, so that when...
    ty = g.types()  # ... g.set_row/g.set_qubit is called, these things update directly to reflect that
    phases = g.phases()
    c = Circuit(g.qubit_count())
    czs_saved = 0

    gadgets = {}
    for v in g.vertices():
        if g.vertex_degree(v) == 1 and v not in g.inputs and v not in g.outputs:
            n = list(g.neighbours(v))[0]
            gadgets[n] = v
    
    qubit_map = dict()
    frontier = []
    for o in g.outputs:
        v = list(g.neighbours(o))[0]
        frontier.append(v)
        qubit_map[v] = qs[o]
#         if len(list(g.neighbours(v))) == 2:
#             for w in g.neighbours(v):
#                 if len(list(g.neighbours(w))) == 2 and any(i in g.inputs for i in g.neighbours(w)) and phases[w] == 0:
#                     finished.add(qubit_map[v])
    
    frontier.sort(key=lambda v: qubit_map[v])
    
    while True:
        # preprocessing
        for v in frontier: # First removing single qubit gates
            q = qubit_map[v]
            b = [w for w in g.neighbours(v) if w in g.outputs][0]
            e = g.edge(v,b)
            if g.edge_type(e) == 2: # Hadamard edge
                c.add_gate("HAD",q)
                g.set_edge_type(e,1)
            if phases[v]: 
                c.add_gate("ZPhase", q, phases[v])
                g.set_phase(v,0)
        # And now on to CZ gates
        cz_mat = Mat2([[0 for i in range(g.qubit_count())] for j in range(g.qubit_count())])
        for v in frontier:
            for w in list(g.neighbours(v)):
                if w in frontier:
                    cz_mat.data[qubit_map[v]][qubit_map[w]] = 1
                    cz_mat.data[qubit_map[w]][qubit_map[v]] = 1
                    g.remove_edge(g.edge(v,w))
        
        # TODO map CZs
        for i in range(g.qubit_count()):
            for j in range(i+1,g.qubit_count()):
                if cz_mat.data[i][j]==1:
                    c.add_gate("CZ",i,j)
        
        # Now we can proceed with the actual extraction
        # First make sure that frontier is connected in correct way to inputs
        neighbours = set()
        finished = set()
        for v in frontier.copy():
            d = [w for w in g.neighbours(v) if w not in g.outputs]
            if any(w in g.inputs for w in d): #frontier vertex v is connected to an input
                if len(d) == 1: # Only connected to input, remove from frontier
                    finished.add(qubit_map[v])
                    #frontier.remove(v)
                    #continue
                # We disconnect v from the input b via a new spider
                b = [w for w in d if w in g.inputs][0]
                q = qs[b]
                r = rs[b]
                w = g.add_vertex(1,q,r+1)
                e = g.edge(v,b)
                et = g.edge_type(e)
                g.remove_edge(e)
                g.add_edge((v,w),2)
                g.add_edge((w,b),3-et)
                d.remove(b)
                d.append(w)
            else:
                if len(d) == 1:
                    w = d[0]
                    if len(g.neighbours(w)) == 2 and phases[w] == 0 and any(i in g.inputs for i in g.neighbours(w)):
                        finished.add(qubit_map[v])
            neighbours.update(d)
        
        if len(frontier) == len(finished): break # No more vertices to be processed. We are done.
        
        # First we check if there is a phase gadget in the way
        removed_gadget = False
        for w in neighbours:
            if w not in gadgets: continue
            for v in g.neighbours(w):
                if v in frontier:
                    apply_rule(g,pivot,[(w,v,[],[o for o in g.neighbours(v) if o in g.outputs])])
                    i = frontier.index(v)
                    frontier[i] = w
                    del gadgets[w]
                    #frontier.append(w)
                    qubit_map[w] = qubit_map[v]
                    removed_gadget = True
                    break
        if removed_gadget: # There was indeed a gadget in the way. Go back to the top
            continue
            
        neighbours = list(neighbours)
        m = bi_adj(g,neighbours,frontier)
        if all(sum(row)!=1 for i, row in enumerate(m.data) if i not in finished): # No easy vertex
            perm = column_optimal_swap(m)
            perm = {v:k for k,v in perm.items()}
            neighbours = [neighbours[perm[i]] for i in range(len(neighbours))]
            m = bi_adj(g, neighbours, frontier)
            cnots = steiner_cnots(m, architecture)
            cnots = filter_duplicate_cnots(cnots) # Since the matrix is not square, the algorithm sometimes introduces duplicates
            if not quiet: print("Gaussian elimination with", len(cnots), "CNOTs")
            # We now have a set of CNOTs that suffice to extract at least one vertex.
            m2 = m.copy()
            for cnot in cnots:
                m2.row_add(cnot.target,cnot.control)
            extractable = set()
            for i, row in enumerate(m2.data):
                if sum(row) == 1:
                    extractable.add(i)
            # We now know which vertices are extractable, and hence the CNOTs on qubits that do not involved
            # these vertices aren't necessary.
            # So first, we get rid of all the CNOTs that happen in the Gaussian elimination after 
            # all the extractable vertices have become extractable
            m2 = m.copy()
            for count, cnot in enumerate(cnots):
                if sum(1 for row in m2.data if sum(row)==1) == len(extractable): #extractable rows equal to maximum
                    cnots = cnots[:count] # So we do not need the remainder of the CNOTs
                    break
                m2.row_add(cnot.target, cnot.control)
            # We now recalculate which vertices were extractable, because the deleted cnots
            # might have acted to swap this vertex around some.
            extractable = set() 
            for i, row in enumerate(m2.data):
                if sum(row) == 1:
                    extractable.add(i)
            # And now we try to get rid of some more CNOTs, that can be commuted to the end of the CNOT circuit
            # without changing extractability.
            necessary_cnots = []
            blocked = {i:'A' for i in extractable} # 'A' stands for "blocked for All".
            for cnot in reversed(cnots):
                if cnot.target not in blocked and cnot.control not in blocked: continue #CNOT not needed
                should_add = False
                if cnot.target in blocked and blocked[cnot.target] != 'R': 
                    should_add = True
                    blocked[cnot.target] = 'A'
                if cnot.control in blocked and blocked[cnot.control] != 'G':
                    should_add = True
                    blocked[cnot.control] = 'A'
                if cnot.control in extractable: should_add = True
                if cnot.target in extractable: should_add = True
                if not should_add: continue
                necessary_cnots.append(cnot)
                if cnot.control not in blocked: blocked[cnot.control] = 'G' # 'G' stands for Green
                if cnot.target not in blocked: blocked[cnot.target] = 'R' # 'R' stands for Red
            if not quiet: print("Actual realization required", len(necessary_cnots), "CNOTs")
            cnots = []
            for cnot in reversed(necessary_cnots):
                m.row_add(cnot.target,cnot.control)
                cnots.append(CNOT(qubit_map[frontier[cnot.control]],qubit_map[frontier[cnot.target]]))
            connectivity_from_biadj(g,m,neighbours,frontier)
        else:
            if not quiet: print("Simple vertex")
            cnots = []
        good_verts = dict()
        
        for i, row in enumerate(m.data):
            if sum(row) == 1:
                v = frontier[i]
                w = neighbours[[j for j in range(len(row)) if row[j]][0]]
                #if len(list(g.neighbours(w))) == 2 and any(z in g.inputs for z in g.neighbours(w)) and phases[w] == 0:
                #    finished.add(qubit_map[v])
                #else: 
                good_verts[v] = w
        if len(good_verts) <= len(finished):# and len(finished) != len(frontier): 
            raise Exception("No extractable vertex found. Something went wrong")
        hads = []
        for v,w in good_verts.items(): # Update frontier vertices
            if len(list(g.neighbours(w))) == 2 and phases[w] == 0 and any(i in g.inputs for i in g.neighbours(w)): continue
            hads.append(qubit_map[v])
            #c.add_gate("HAD",qubit_map[v])
            i = frontier.index(v)
            frontier[i] = w
            qubit_map[w] = qubit_map[v]
            b = [o for o in g.neighbours(v) if o in g.outputs][0]
            g.remove_vertex(v)
            g.add_edge((w,b))
            #frontier.remove(v)
            #frontier.append(w)
        if not quiet: print("Vertices extracted:", len(good_verts))
        for cnot in cnots: c.add_gate(cnot)
        for h in hads: c.add_gate("HAD",h)

    # Outside of loop. Finish up the permutation
    id_simp(g,quiet=True) # Now the graph should only contain inputs and outputs
    swap_map = {}
    leftover_swaps = False
    for v in g.outputs: # Finally, check for the last layer of Hadamards, and see if swap gates need to be applied.
        q = qs[v]
        i = list(g.neighbours(v))[0]
        if i not in g.inputs: 
            raise TypeError("Algorithm failed: Not fully reducable")
            return c
        if g.edge_type(g.edge(v,i)) == 2:
            c.add_gate("HAD", q)
            g.set_edge_type(g.edge(v,i),1)
        if qs[i] != q: leftover_swaps = True
        swap_map[q] = qs[i]
    if leftover_swaps: 
        for t1, t2 in permutation_as_swaps(swap_map):
            c.add_gate("SWAP", t1, t2)
    # Since we were extracting from right to left, we reverse the order of the gates
    c.gates = list(reversed(c.gates))
    return c